In [ ]:
import psycopg2
import pandas as pd
import os
import subprocess
from psycopg2 import sql
from sqlalchemy import create_engine

# Database configuration
db_name = "fitness"
password = "david4e<3"  
user = "postgres"
host = "localhost"
port = "5432"

#Step 1: Create Database 
conn = psycopg2.connect(
    dbname="postgres",
    user=user,
    password=password,
    host=host,
    port=port
)
conn.autocommit = True
cur = conn.cursor()

# Drop database if it already exists
cur.execute(sql.SQL("DROP DATABASE IF EXISTS {}").format(sql.Identifier(db_name)))
# Create a new database
cur.execute(sql.SQL("CREATE DATABASE {}").format(sql.Identifier(db_name)))

cur.close()
conn.close() 





In [ ]:
#Step 2: Create Tables 
conn = psycopg2.connect(
    dbname=db_name,
    user=user,
    password=password,
    host=host,
    port=port
)
cur = conn.cursor()

with open("star.txt", "r", encoding="utf-8") as file:
    sql_queries = file.read()

cur.execute(sql_queries)
conn.commit()
cur.close()
conn.close()

print("✅ Tables created successfully!")

print(sql_queries)


In [74]:
df  = pd.read_csv("categorized_fitlife_data.csv")
df.head()

,participant_id,date,age,gender,height_cm,weight_kg,activity_type,duration_minutes,intensity,calories_burned,...,resting_heart_rate,blood_pressure_systolic,blood_pressure_diastolic,health_condition,smoking_status,fitness_level,pressure_systolic_category,pressure_diastolic_category,heart_rate_category,resting_heart_rate_category
0,1,01/01/2024,56,F,165.3,53.7,Dancing,41,Low,3.3,...,69.5,110.7,72.9,None,Never,0.04,Normal systolic pressure,Normal blood pressure diastolic,High heart rate,Normal resting heart rate
1,1,04/01/2024,56,F,165.3,53.9,Swimming,28,Low,2.9,...,69.5,110.7,72.9,None,Never,0.07,Normal systolic pressure,Normal blood pressure diastolic,High heart rate,Normal resting heart rate
2,1,05/01/2024,56,F,165.3,54.2,Swimming,21,Medium,2.6,...,69.5,110.7,72.9,None,Never,0.09,Normal systolic pressure,Normal blood pressure diastolic,High heart rate,Normal resting heart rate
3,1,07/01/2024,56,F,165.3,54.4,Weight Training,99,Medium,10.7,...,69.5,110.7,72.9,None,Never,0.21,Normal systolic pressure,Normal blood pressure diastolic,High heart rate,Normal resting heart rate
4,1,09/01/2024,56,F,165.3,54.7,Swimming,100,Medium,12.7,...,69.5,110.7,72.9,None,Never,0.33,Normal systolic pressure,Normal blood pressure diastolic,High heart rate,Normal resting heart rate


In [ ]:
#Step 3: Load & Preprocess Data
df.rename(columns={
    "participant_id": "person_id",
    "date": "time_id",
   "hours_sleep": "hours_of_sleep"
}, inplace=True)

df["time_id"] = pd.to_datetime(df["time_id"], dayfirst=True, errors='coerce')

numeric_columns = ["age", "height_cm", "weight_kg", "duration_minutes", "bmi", "avg_heart_rate",
                   "resting_heart_rate", "blood_pressure_systolic", "blood_pressure_diastolic",
                   "stress_level", "hours_of_sleep", "hydration_level", "calories_burned", "daily_steps"]
df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors='coerce')

df.dropna(subset=numeric_columns, inplace=True)

print(df.isnull().sum())
engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{db_name}')


person_id                      0
time_id                        0
age                            0
gender                         0
height_cm                      0
weight_kg                      0
activity_type_id               0
duration_minutes               0
intensity                      0
calories_burned                0
avg_heart_rate                 0
hours_of_sleep                 0
stress_level                   0
daily_steps                    0
hydration_level                0
bmi                            0
resting_heart_rate             0
blood_pressure_systolic        0
blood_pressure_diastolic       0
health_condition               0
smoking_status                 0
fitness_level                  0
pressure_systolic_category     0
pressure_diastolic_category    0
heart_rate_category            0
resting_heart_rate_category    0
dtype: int64


In [ ]:
# Insert data in person
# Connect to the database
conn = psycopg2.connect(dbname=db_name, user=user, password=password, host=host, port=port)
cur = conn.cursor()

try:
    cur.execute("ALTER TABLE person ADD CONSTRAINT unique_person_id UNIQUE (person_id);")
except psycopg2.errors.DuplicateTable:
    conn.rollback()  # If constraint exists, rollback the transaction

# Prepare the data for insertion
df_person = df[["person_id", "age", "gender", "height_cm", "weight_kg", "bmi", "smoking_status"]].drop_duplicates()


# Convert data types
df_person = df_person.astype({
    "person_id": int,
    "age": int,
    "gender": str,
    "height_cm": float,
    "weight_kg": float,
    "bmi": float,
    "smoking_status": str
})

# Insert data into person table
for row in df_person.itertuples(index=False):
    cur.execute("""
        INSERT INTO person (person_id, age, gender, height_cm, weight_kg, bmi, smoking_status)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
        ON CONFLICT (person_id) DO UPDATE SET
            age = EXCLUDED.age,
            gender = EXCLUDED.gender,
            height_cm = EXCLUDED.height_cm,
            weight_kg = EXCLUDED.weight_kg,
            bmi = EXCLUDED.bmi,
            smoking_status = EXCLUDED.smoking_status;
    """, row)

conn.commit()
cur.close()
conn.close()


   person_id  age gender  height_cm  weight_kg   bmi smoking_status
0          1   56      F      165.3       53.7  19.6          Never
1          1   56      F      165.3       53.9  19.6          Never
2          1   56      F      165.3       54.2  19.6          Never
3          1   56      F      165.3       54.4  19.6          Never
4          1   56      F      165.3       54.7  19.6          Never
✅ Finished inserting into Person table!


In [ ]:
df_timeinfo = df[["time_id"]]
df_timeinfo["month"] = df_timeinfo["time_id"].dt.month
df_timeinfo["season"] = df_timeinfo["month"].map({ 
    12: "Winter", 1: "Winter", 2: "Winter",
    3: "Spring", 4: "Spring", 5: "Spring",
    6: "Summer", 7: "Summer", 8: "Summer",
    9: "Autumn", 10: "Autumn", 11: "Autumn"
})
df_timeinfo["weekend"] = df_timeinfo["time_id"].dt.weekday >= 5 

conn = psycopg2.connect(dbname=db_name, user=user, password=password, host=host, port=port)
cur = conn.cursor()

# Insert into timeinfo table
for row in df_timeinfo.itertuples(index=False):
    cur.execute("""
        INSERT INTO timeinfo (time_id, month, season, weekend)
        VALUES (%s, %s, %s, %s)
        ON CONFLICT (time_id) DO NOTHING;
    """, row)

conn.commit()
cur.close()
conn.close()


C:\Users\Acer\AppData\Local\Temp\ipykernel_11332\189307625.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_timeinfo["month"] = df_timeinfo["time_id"].dt.month
C:\Users\Acer\AppData\Local\Temp\ipykernel_11332\189307625.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_timeinfo["season"] = df_timeinfo["month"].map({
C:\Users\Acer\AppData\Local\Temp\ipykernel_11332\189307625.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

✅ Finished inserting into timeinfo!


In [ ]:
df_health = df[[
    "person_id", "avg_heart_rate", "resting_heart_rate",
    "blood_pressure_diastolic", "blood_pressure_systolic", "stress_level",
    "hours_of_sleep", "hydration_level"
]].drop_duplicates()

df_health.rename(columns={"person_id": "health_id"}, inplace=True)

df_health["person_id"] = df_health["health_id"] 

conn = psycopg2.connect(dbname=db_name, user=user, password=password, host=host, port=port)
cur = conn.cursor()

for row in df_health.itertuples(index=False):
    cur.execute("""
        INSERT INTO health_ (health_id, avg_heart_rate, resting_heart_rate, 
                             blood_pressure_diastolic, blood_pressure_systolic, stress_level, 
                             hours_of_sleep, hydration_level, person_id)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
        ON CONFLICT (health_id) DO NOTHING;
    """, row)

conn.commit()
cur.close()
conn.close()


Finished inserting into health_!


In [ ]:
df = pd.DataFrame({
    'activity_type': ['Dancing', 'Swimming', 'Weight Training','HIIT','Running', 'Walking', 'Tennis', 'Basketball','Yoga', 'Cycling'],
    'intensity': ['Low', 'Low', 'High', 'Medium', 'Medium', 'Low', 'High', 'High', 'Low', 'Medium']
})

# Map activity_type to activity_type_id
activity_mapping = {
    "Dancing": 1,
    "Swimming": 2,
    "Weight Training": 3,
    "HIIT": 4,
    "Running": 5,
    "Walking": 6,
    "Tennis": 7,
    "Basketball": 8,
    "Yoga": 9,
    "Cycling": 10,
}

df['activity_type_id'] = df['activity_type'].map(activity_mapping)

df.dropna(subset=['activity_type', 'intensity'], inplace=True)

conn = psycopg2.connect(dbname=db_name, user=user, password=password, host=host, port=port)
cur = conn.cursor()

# Prepare the data for insertion (Activity Type Data)
activity_type_data = [(row['activity_type_id'], row['activity_type'], row['intensity']) for _, row in df.iterrows()]

#Insert into the database
for row in activity_type_data:
    cur.execute("""
        INSERT INTO activity_type (activity_type_id, activity_type, intensity)
        VALUES (%s, %s, %s)
        ON CONFLICT (activity_type_id) DO NOTHING;
    """, row)

conn.commit()

cur.close()
conn.close()


     activity_type intensity  activity_type_id
0          Dancing       Low                 1
1         Swimming       Low                 2
2  Weight Training      High                 3
3             HIIT    Medium                 4
4          Running    Medium                 5
5          Walking       Low                 6
6           Tennis      High                 7
7       Basketball      High                 8
8             Yoga       Low                 9
9          Cycling    Medium                10
✅ Data inserted into activity_type table successfully!


In [ ]:

df_activity = df[[
    "participant_id", "date", "activity_type", "duration_minutes", 
    "calories_burned", "daily_steps", "health_condition", "smoking_status"
]].drop_duplicates()

activity_mapping = {
    "Dancing": 1,
    "Swimming": 2,
    "Weight Training": 3,
    "HIIT": 4,
    "Running": 5,
    "Walking": 6,
    "Tennis": 7,
    "Basketball": 8,
    "Yoga": 9,
    "Cycling": 10,
}

df_activity['activity_type_id'] = df_activity['activity_type'].map(activity_mapping)

conn = psycopg2.connect(dbname=db_name, user=user, password=password, host=host, port=port)
cur = conn.cursor()

for row in df_activity.itertuples(index=False):
    participant_id = row[0]  
    date = row[1]  # Second column is date
    activity_type = row[2]  # Third column is activity_type
    duration_minutes = row[3]  # Fourth column is duration_minutes
    calories_burned = row[4]  # Fifth column is calories_burned
    daily_steps = row[5]  # Sixth column is daily_steps
    health_condition = row[6]  # Seventh column is health_condition
    smoking_status = row[7]  # Eighth column is smoking_status
    activity_type_id = row[8]  # Ninth column is activity_type_id

    # Execute the insert query
    cur.execute("""
        INSERT INTO activity (
            person_id, activity_type_id, time_id, calories_burned, 
            duration, daily_steps, activity_type
        )
        VALUES (%s, %s, %s, %s, %s, %s, %s)
        ON CONFLICT (activity_id) DO NOTHING;
    """, (
        participant_id, 
        activity_type_id, 
        date,  
        calories_burned,
        duration_minutes,
        daily_steps,
        activity_type  
    ))

# Commit the transaction
conn.commit()

# Close the cursor and connection
cur.close()
conn.close()

print("✅ Finished inserting into Activity table!")


✅ Finished inserting into Activity table!
